In [1]:
from selenium import webdriver
#Para generar eventos de click
from selenium.webdriver.common.keys import Keys
#Para demoras dinámicas
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
from time import time
from datetime import datetime
#Para convertir listas a dataframe
import pandas as pd

In [ ]:
url = 'https://www.portalinmobiliario.com/MLC-618072768-amplia-casa-esquina-en-villa-el-vergel-maipu-_JM#position=1&search_layout=stack&type=item&tracking_id=aceba5bc-3b97-4ab3-85bc-1f839fddf8cc'

In [277]:
propiedad_arriendo = requests.get(url)

In [395]:
propiedad_arriendo.max_redirects = 60

In [278]:
propiedad_arriendo.status_code

200

In [279]:
s_prop_arriendo = BeautifulSoup(propiedad_arriendo.text, 'lxml')

In [27]:
tabla.get_text()

'122 m²'

In [30]:
valores_tabla = [tabla.text for tabla in tablas]
valores_tabla

['122 m²', '84 m²', '3', '3', '2', '2', 'O', '0 CLP']

In [33]:
datos_tabla = [tabla.text for tabla in tablas]
datos_tabla

['Superficie total',
 'Superficie útil',
 'Dormitorios',
 'Baños',
 'Estacionamientos',
 'Cantidad de pisos',
 'Orientación',
 'Gastos comunes']

In [34]:
dict_from_list = dict(zip(datos_tabla, valores_tabla))
print(dict_from_list)

{'Superficie total': '122 m²', 'Superficie útil': '84 m²', 'Dormitorios': '3', 'Baños': '3', 'Estacionamientos': '2', 'Cantidad de pisos': '2', 'Orientación': 'O', 'Gastos comunes': '0 CLP'}


In [35]:
#Extramos el título
s_prop_prueba.find('h1', attrs={'class':'ui-pdp-title'}).text


'Amplia Casa Esquina En Villa El Vergel, Maipú.'

In [55]:
#Extraemos valor A
s_prop_prueba.find('div', attrs={'class':'ui-pdp-price__second-line'}).find(attrs={'class':'price-tag-fraction'}).text


'3.900'

In [54]:
#Extraemos valor B
s_prop_prueba.find('div', attrs={'class':'ui-pdp-price__subtitles'}).find(attrs={'class':'price-tag-fraction'}).text



'115.868.342'

In [2]:
def obtener_info_arriendo(s_prop_arriendo):
    '''
    Función que retorna un diccionario con las variables de interés de cada una de las propiedades de la página.
    Nota: El valor de la propiedad no se representará en la misma moneda por cada columna, 
    esto es algo que se debe limpiar después en la próxima etapa.
    '''
    data={}
    
    #Indicamos la url
    #try:
      #  data['url_prop']= url
   # except:
      #  data['url_prop']=url
        
    #Extramos el título
    try:
        data['titulo'] = s_prop_arriendo.find('h1', attrs={'class':'ui-pdp-title'}).text
    except:
        data['titulo'] = None
    
    #Extraemos tipo de propiedad
    try:
        if 'Departamento' in s_prop_arriendo.find('span', attrs={'class':'ui-pdp-subtitle'}).text:
            data['tipo_propiedad'] = 'Departamento'
        elif 'Casa' in s_prop_arriendo.find('span', attrs={'class':'ui-pdp-subtitle'}).text: 
            data['tipo_propiedad'] = 'Casa'
        else:
            data['tipo_propiedad'] = ''
    except:
        data['tipo_propiedad'] = None
    #Extraemos tiempo de publicación
    try:
        data['dias_publicacion'] = s_prop_arriendo.find('p', attrs={'class':'ui-pdp-color--GRAY ui-pdp-size--XSMALL ui-pdp-family--REGULAR ui-pdp-header__bottom-subtitle'}).text 
    except:
        data['dias_publicacion'] = None
    #Indicamos fecha de hoy
    try:
        data['fecha_extracción']=datetime.now().strftime('%Y-%m-%d')
    except:
        data['fecha_extracción']=None
    #Extraemos ubicación
    try:
        data['ubicacion']= s_prop_arriendo.find('div', attrs={'class':'ui-vip-location'}).find(attrs={'class':'ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--REGULAR ui-pdp-media__title'}).text
    except:
        data['ubicacion']= None
    #Extraemos valor A
    try:
        valor_a = s_prop_arriendo.find('div', attrs={'class':'ui-pdp-price__second-line'}).find(attrs={'class':'price-tag-fraction'}).text
        data['valor_A']=valor_a
    except:
        data['valor_A']=None
    #Extraemos valor B
    try:
        valor_b = s_prop_arriendo.find('div', attrs={'class':'ui-pdp-price__subtitles'}).find(attrs={'class':'price-tag-fraction'}).text
        data['valor_B']=valor_b
    except:
        data['valor_B']=None
    #Extraemos valor m2 en propiedad
    try:
        data['valor_m2_prop']=s_prop_arriendo.find_all('p', attrs={'class':'ui-pdp-color--BLACK ui-pdp-size--XSMALL ui-pdp-family--REGULAR ui-pdp-price-comparison__extra-info-element-value'})[0].text
    except:
        data['valor_m2_prop']=None
    #Extraemos valor m2 en comuna
    try:
        data['valor_m2_comuna']=s_prop_arriendo.find_all('p', attrs={'class':'ui-pdp-color--BLACK ui-pdp-size--XSMALL ui-pdp-family--REGULAR ui-pdp-price-comparison__extra-info-element-value'})[1].text
    except:
        data['valor_m2_comuna']=None
    #datos de tabla
    try:
        datos_tabla = s_prop_arriendo.find('table', attrs={'class':'andes-table'}).find_all('th')
        dato = datos_tabla[0]
        datos = [dato.text for dato in datos_tabla]
    except:
        datos =None
    #valores de tabla
    try:
        valores_tabla = s_prop_arriendo.find('table', attrs={'class':'andes-table'}).find_all('td')
        valor = valores_tabla[0]
        valores = [valor.text for valor in valores_tabla]
    except:
        valores =None
    tabla = dict(zip(datos, valores))
    
    #Unimos todo
    data.update(tabla)
    
    return data

    
    

In [294]:
lista_url_prop =['https://www.portalinmobiliario.com/MLC-618072768-amplia-casa-esquina-en-villa-el-vergel-maipu-_JM#position=1&search_layout=stack&type=item&tracking_id=aceba5bc-3b97-4ab3-85bc-1f839fddf8cc','https://www.portalinmobiliario.com/arriendo/casa/buin-metropolitana/5532735-barrio-villaseca-uda#position=2&search_layout=stack&type=item&tracking_id=c79084e5-323d-4b53-9116-17e6a4b51ce9']

In [3]:
def obtener_info(lista_url_prop):
    hora_start = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
    start_time = time() #Comienza conteo de tiempo
    print(f'El scraping se comienza a ejecutar a las {hora_start}')
    prop = 1 #Nº de propiedad con que comienza la descarga (sólo referencial, no condiciona la programación)
    info = []
    print(f'Se encontraron {len(lista_url_prop)} propiedades en la lista')
    lista=lista_url_prop
    try:
        for url in lista:
            try:
                prop_arriendo = requests.get(url)
                prop_arriendo.max_redirects = 30
                print(f'Quedan {len(lista_url_prop)-prop+1} propiedades por escrapear')#
                print(f'Inicializando scraping de la propiedad nº{prop}...')
                try:
                    if prop_arriendo.status_code ==200:
                        s_prop_arriendo = BeautifulSoup(prop_arriendo.text, 'lxml')
                        detalle_arriendo = obtener_info_arriendo(s_prop_arriendo)
                        
                        print(f'Se termina scraping a propiedad...')

                    else:
                        print(f'No fue posible scrapear la propiedad nº{prop}')
                        pass

                except:
                    print(f'Hubo una interrupción al escrapear la propiedad nº{prop}...')
                    #print(f'Se devolveran los valores hasta ahora')
                    pass

            

            except Exception as e:
                print(f'Hubo un error al scrapear la propiedad nº{prop}, el error fue: {e}')
                #print(f'Se devolveran los valores hasta ahora')
                #print(f'Se escrapearon {prop-1} propiedades')
                #hora_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
                #print(f'El scraping se comienzó a ejecutar a las {hora_start}')
                #print(f'El scraping se termina de ejecutar a las {hora_end}')
                #elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
                #print("Tiempo de ejecución: %.3f seconds." % elapsed_time) 
                pass
                #return info
            except:
                print(f'Hubo una interrupción al escrapear la propiedad nº{prop}')
                #print(f'Se devolveran los valores hasta ahora')
                pass
                
            prop +=1
            info.append(detalle_arriendo) 
            
        print(f'Se escrapearon {prop-1} propiedades')
        hora_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
        print(f'El scraping se comienzó a ejecutar a las {hora_start}')
        print(f'El scraping se termina de ejecutar a las {hora_end}')
        elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
        print("Tiempo de ejecución: %.3f seconds." % elapsed_time)
    except:
        print(f'Se devolveran los valores hasta ahora')
        return info
     
        
    return info
        
    

In [364]:
#obtener_info(lista_url_prop)

In [304]:
#Leer csv con links de arriendos casa
lista_url_arriendos_casa = pd.read_csv("Link_casa_arriendo.csv", delimiter=";")


In [305]:
#Convertir en lista campo de links (casa)
lista_url_casa=lista_url_arriendos_casa['links_intermedios'].tolist()


In [4]:
#Leer csv con links de arriendos depto
lista_url_arriendos_depa = pd.read_csv("Links_depa_arriendo.csv", delimiter=";")

In [6]:
#Convertir en lista campo de links (depa)
lista_url_depa=lista_url_arriendos_depa['links_intermedios'].tolist()

In [365]:
#lista_url_casa

In [7]:
info_depa=obtener_info(lista_url_depa[5693:])

El scraping se comienza a ejecutar a las 2021-07-02 00:10:29
Se encontraron 8996 propiedades en la lista
Quedan 8996 propiedades por escrapear
Inicializando scraping de la propiedad nº1...
Se termina scraping a propiedad...
Quedan 8995 propiedades por escrapear
Inicializando scraping de la propiedad nº2...
Se termina scraping a propiedad...
Quedan 8994 propiedades por escrapear
Inicializando scraping de la propiedad nº3...
Se termina scraping a propiedad...
Quedan 8993 propiedades por escrapear
Inicializando scraping de la propiedad nº4...
Se termina scraping a propiedad...
Quedan 8992 propiedades por escrapear
Inicializando scraping de la propiedad nº5...
Se termina scraping a propiedad...
Quedan 8991 propiedades por escrapear
Inicializando scraping de la propiedad nº6...
Se termina scraping a propiedad...
Quedan 8990 propiedades por escrapear
Inicializando scraping de la propiedad nº7...
Se termina scraping a propiedad...
Quedan 8989 propiedades por escrapear
Inicializando scraping d

Quedan 8928 propiedades por escrapear
Inicializando scraping de la propiedad nº69...
Se termina scraping a propiedad...
Quedan 8927 propiedades por escrapear
Inicializando scraping de la propiedad nº70...
Se termina scraping a propiedad...
Quedan 8926 propiedades por escrapear
Inicializando scraping de la propiedad nº71...
Se termina scraping a propiedad...
Quedan 8925 propiedades por escrapear
Inicializando scraping de la propiedad nº72...
Se termina scraping a propiedad...
Quedan 8924 propiedades por escrapear
Inicializando scraping de la propiedad nº73...
Se termina scraping a propiedad...
Quedan 8923 propiedades por escrapear
Inicializando scraping de la propiedad nº74...
Se termina scraping a propiedad...
Quedan 8922 propiedades por escrapear
Inicializando scraping de la propiedad nº75...
Se termina scraping a propiedad...
Quedan 8921 propiedades por escrapear
Inicializando scraping de la propiedad nº76...
Se termina scraping a propiedad...
Quedan 8920 propiedades por escrapear
In

Quedan 8860 propiedades por escrapear
Inicializando scraping de la propiedad nº137...
Se termina scraping a propiedad...
Quedan 8859 propiedades por escrapear
Inicializando scraping de la propiedad nº138...
Se termina scraping a propiedad...
Quedan 8858 propiedades por escrapear
Inicializando scraping de la propiedad nº139...
Se termina scraping a propiedad...
Quedan 8857 propiedades por escrapear
Inicializando scraping de la propiedad nº140...
Se termina scraping a propiedad...
Quedan 8856 propiedades por escrapear
Inicializando scraping de la propiedad nº141...
Se termina scraping a propiedad...
Quedan 8855 propiedades por escrapear
Inicializando scraping de la propiedad nº142...
Se termina scraping a propiedad...
Quedan 8854 propiedades por escrapear
Inicializando scraping de la propiedad nº143...
Se termina scraping a propiedad...
Quedan 8853 propiedades por escrapear
Inicializando scraping de la propiedad nº144...
Se termina scraping a propiedad...
Quedan 8852 propiedades por escr

Quedan 8792 propiedades por escrapear
Inicializando scraping de la propiedad nº205...
Se termina scraping a propiedad...
Quedan 8791 propiedades por escrapear
Inicializando scraping de la propiedad nº206...
Se termina scraping a propiedad...
Quedan 8790 propiedades por escrapear
Inicializando scraping de la propiedad nº207...
Se termina scraping a propiedad...
Quedan 8789 propiedades por escrapear
Inicializando scraping de la propiedad nº208...
Se termina scraping a propiedad...
Quedan 8788 propiedades por escrapear
Inicializando scraping de la propiedad nº209...
Se termina scraping a propiedad...
Quedan 8787 propiedades por escrapear
Inicializando scraping de la propiedad nº210...
Se termina scraping a propiedad...
Quedan 8786 propiedades por escrapear
Inicializando scraping de la propiedad nº211...
Se termina scraping a propiedad...
Quedan 8785 propiedades por escrapear
Inicializando scraping de la propiedad nº212...
Se termina scraping a propiedad...
Quedan 8784 propiedades por escr

Quedan 8724 propiedades por escrapear
Inicializando scraping de la propiedad nº273...
Se termina scraping a propiedad...
Quedan 8723 propiedades por escrapear
Inicializando scraping de la propiedad nº274...
Se termina scraping a propiedad...
Quedan 8722 propiedades por escrapear
Inicializando scraping de la propiedad nº275...
Se termina scraping a propiedad...
Quedan 8721 propiedades por escrapear
Inicializando scraping de la propiedad nº276...
Se termina scraping a propiedad...
Quedan 8720 propiedades por escrapear
Inicializando scraping de la propiedad nº277...
Se termina scraping a propiedad...
Quedan 8719 propiedades por escrapear
Inicializando scraping de la propiedad nº278...
Se termina scraping a propiedad...
Quedan 8718 propiedades por escrapear
Inicializando scraping de la propiedad nº279...
Se termina scraping a propiedad...
Quedan 8717 propiedades por escrapear
Inicializando scraping de la propiedad nº280...
Se termina scraping a propiedad...
Quedan 8716 propiedades por escr

...

Quedan 297 propiedades por escrapear
Inicializando scraping de la propiedad nº8700...
Se termina scraping a propiedad...
Quedan 296 propiedades por escrapear
Inicializando scraping de la propiedad nº8701...
Se termina scraping a propiedad...
Quedan 295 propiedades por escrapear
Inicializando scraping de la propiedad nº8702...
Se termina scraping a propiedad...
Quedan 294 propiedades por escrapear
Inicializando scraping de la propiedad nº8703...
Se termina scraping a propiedad...
Quedan 293 propiedades por escrapear
Inicializando scraping de la propiedad nº8704...
Se termina scraping a propiedad...
Quedan 292 propiedades por escrapear
Inicializando scraping de la propiedad nº8705...
Se termina scraping a propiedad...
Quedan 291 propiedades por escrapear
Inicializando scraping de la propiedad nº8706...
Se termina scraping a propiedad...
Quedan 290 propiedades por escrapear
Inicializando scraping de la propiedad nº8707...
Se termina scraping a propiedad...
Quedan 289 propiedades por escra

Quedan 229 propiedades por escrapear
Inicializando scraping de la propiedad nº8768...
Se termina scraping a propiedad...
Quedan 228 propiedades por escrapear
Inicializando scraping de la propiedad nº8769...
Se termina scraping a propiedad...
Quedan 227 propiedades por escrapear
Inicializando scraping de la propiedad nº8770...
Se termina scraping a propiedad...
Quedan 226 propiedades por escrapear
Inicializando scraping de la propiedad nº8771...
Se termina scraping a propiedad...
Quedan 225 propiedades por escrapear
Inicializando scraping de la propiedad nº8772...
Se termina scraping a propiedad...
Quedan 224 propiedades por escrapear
Inicializando scraping de la propiedad nº8773...
Se termina scraping a propiedad...
Quedan 223 propiedades por escrapear
Inicializando scraping de la propiedad nº8774...
Se termina scraping a propiedad...
Quedan 222 propiedades por escrapear
Inicializando scraping de la propiedad nº8775...
Se termina scraping a propiedad...
Quedan 221 propiedades por escra

Quedan 161 propiedades por escrapear
Inicializando scraping de la propiedad nº8836...
Se termina scraping a propiedad...
Quedan 160 propiedades por escrapear
Inicializando scraping de la propiedad nº8837...
Se termina scraping a propiedad...
Quedan 159 propiedades por escrapear
Inicializando scraping de la propiedad nº8838...
Se termina scraping a propiedad...
Quedan 158 propiedades por escrapear
Inicializando scraping de la propiedad nº8839...
Se termina scraping a propiedad...
Quedan 157 propiedades por escrapear
Inicializando scraping de la propiedad nº8840...
Se termina scraping a propiedad...
Quedan 156 propiedades por escrapear
Inicializando scraping de la propiedad nº8841...
Se termina scraping a propiedad...
Quedan 155 propiedades por escrapear
Inicializando scraping de la propiedad nº8842...
Se termina scraping a propiedad...
Quedan 154 propiedades por escrapear
Inicializando scraping de la propiedad nº8843...
Se termina scraping a propiedad...
Quedan 153 propiedades por escra

Quedan 93 propiedades por escrapear
Inicializando scraping de la propiedad nº8904...
Se termina scraping a propiedad...
Quedan 92 propiedades por escrapear
Inicializando scraping de la propiedad nº8905...
Se termina scraping a propiedad...
Quedan 91 propiedades por escrapear
Inicializando scraping de la propiedad nº8906...
Se termina scraping a propiedad...
Quedan 90 propiedades por escrapear
Inicializando scraping de la propiedad nº8907...
Se termina scraping a propiedad...
Quedan 89 propiedades por escrapear
Inicializando scraping de la propiedad nº8908...
Se termina scraping a propiedad...
Quedan 88 propiedades por escrapear
Inicializando scraping de la propiedad nº8909...
Se termina scraping a propiedad...
Quedan 87 propiedades por escrapear
Inicializando scraping de la propiedad nº8910...
Se termina scraping a propiedad...
Quedan 86 propiedades por escrapear
Inicializando scraping de la propiedad nº8911...
Se termina scraping a propiedad...
Quedan 85 propiedades por escrapear
Inic

Quedan 24 propiedades por escrapear
Inicializando scraping de la propiedad nº8973...
Se termina scraping a propiedad...
Quedan 23 propiedades por escrapear
Inicializando scraping de la propiedad nº8974...
Se termina scraping a propiedad...
Quedan 22 propiedades por escrapear
Inicializando scraping de la propiedad nº8975...
Se termina scraping a propiedad...
Quedan 21 propiedades por escrapear
Inicializando scraping de la propiedad nº8976...
Se termina scraping a propiedad...
Quedan 20 propiedades por escrapear
Inicializando scraping de la propiedad nº8977...
Se termina scraping a propiedad...
Quedan 19 propiedades por escrapear
Inicializando scraping de la propiedad nº8978...
Se termina scraping a propiedad...
Quedan 18 propiedades por escrapear
Inicializando scraping de la propiedad nº8979...
Se termina scraping a propiedad...
Quedan 17 propiedades por escrapear
Inicializando scraping de la propiedad nº8980...
Se termina scraping a propiedad...
Quedan 16 propiedades por escrapear
Inic

In [12]:
#info_depa

In [9]:
info_depa_5693_14689=info_depa

In [10]:
df = pd.DataFrame(info_depa_5693_14689)
#df

In [ ]:
info_depa_1_1894 #
info_depa_1895_2301 #
info_depa_2302_3566#
info_depa_3567_5692#
nfo_depa_5693_14689#

In [11]:
#Exportar data de arriendos (depa)
df.to_csv('data_arriendo_depa_portal_5.csv')

In [15]:
#Leer csv con resultados
data_casa = pd.read_csv("data_arriendo_casas_portal_1.csv")

,Unnamed: 0,titulo,tipo_propiedad,dias_publicacion,fecha_extracción,ubicacion,valor_A,valor_B,valor_m2_prop,valor_m2_comuna,...,Superficie útil,Dormitorios,Baños,Estacionamientos,Gastos comunes,Antigüedad,Cantidad de pisos,Ambientes,Bodegas,Orientación
0,0,Barrio Villaseca 1 - 300,Casa,Publicado hace 1 año,2021-07-01,"Barrio Villaseca 1 - 300, Villaseca, Buin, RM ...",420.000,NaN,"0,13 UF/m²",NaN,...,60 m²,3.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Arriendo Casa Georgiana,Casa,Publicado hace 3 días,2021-07-01,"El Tanguito 1, El Copihue, Calera De Tango, RM...",1.700.000,NaN,"0,04 UF/m²",NaN,...,500 m²,5.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Piedra Roja / Chicureo (las Bandadas),Casa,Publicado hace 1 año,2021-07-01,"Piedra Roja / Chicureo (las Bandadas), Chicure...",2.700.000,NaN,"0,12 UF/m²","0,08 UF/m²",...,240 m²,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Increible Casa, Las Brisas De Chicureo, Gran T...",Casa,Publicado hace 9 meses,2021-07-01,"Increible Casa, Las Brisas De Chicureo, Gran T...",55,1.634.204,"0,02 UF/m²",NaN,...,280 m²,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Chicureo Oriente - Avenida Del Valle,Casa,Publicado hace 1 año,2021-07-01,"Chicureo Oriente - Avenida Del Valle, Chicureo...",5.475.300,NaN,"0,12 UF/m²",NaN,...,250 m²,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,1329,Casa En Arriendo De 3 Dorm. En Maipú,Casa,Publicado hace 1 día,2021-07-01,"Pasaje Cerro Largo / Av Cuatro Pte., Maipú, RM...",400.000,NaN,0 UF/m²,NaN,...,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1330,1330,Casa En Arriendo En Santiago,Casa,Publicado hace 1 día,2021-07-01,"Sotomayor // Alameda, Santiago, RM (Metropolit...",890,26.444.390,NaN,NaN,...,2169 m²,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1331,1331,Casa Remodelada Calle Escocia,Casa,Publicado hace 1 día,2021-07-01,"Escocia, Las Condes, Chile, Los Dominicos, Las...",2.200.000,NaN,"0,12 UF/m²",NaN,...,300 m²,3.0,3.0,2.0,NaN,NaN,2.0,NaN,NaN,NOSP
1332,1332,Se Arriada Casa Paradero 8 1/2 Pajaritos,Casa,Publicado hace 1 día,2021-07-01,"Av. El Descanso 300 - 600, Maipú, Región Metro...",470.000,NaN,"0,08 UF/m²",NaN,...,65 m²,3.0,1.0,3.0,NaN,15 años,1.0,NaN,NaN,NaN


In [22]:
#Leer csv con resultados
data_casa = pd.read_csv("data_arriendo_casas_portal_1.csv")
data_depa_1 = pd.read_csv("data_arriendo_depa_portal_1.csv")
data_depa_2 = pd.read_csv("data_arriendo_depa_portal_2.csv")
data_depa_3 = pd.read_csv("data_arriendo_depa_portal_3.csv")
data_depa_4 = pd.read_csv("data_arriendo_depa_portal_4.csv")
data_depa_5 = pd.read_csv("data_arriendo_depa_portal_5.csv")

In [24]:
data_total_portal_arriendo=pd.concat([data_casa,data_depa_1,data_depa_2,data_depa_3,data_depa_4,data_depa_5])

In [25]:
data_total_portal_arriendo.to_csv('data_total_arriendo_portal.csv')

**Probando con Selenium**

In [155]:
options = webdriver.ChromeOptions() #Generamos una variable de opción vacía.
options.add_argument('--incognito') #le agregamos a la variable creada un argumento, en este caso de incógnito.
driver = webdriver.Chrome(executable_path= '/Users/danielparedesvillalobos/opt/anaconda3/chromedriver', options=options) #Le agregamos la variable creada al argumento options.

In [ ]:
#driver.get(lista_url_prop)

In [178]:
driver.find_element_by_xpath('//div[@class="ui-pdp-price__second-line"]').find_element_by_xpath('[@class="price-tag-fraction"]').text

'Imperio Inca, Maipú, Chile, Nueva San Martín, Maipú, RM (Metropolitana)'

In [ ]:
def obtener_info_arriendo_selenium(s_prop_arriendo):
    '''
    Función que retorna un diccionario con las variables de interés de cada una de las propiedades de la página.
    Nota: El valor de la propiedad no se representará en la misma moneda por cada columna, 
    esto es algo que se debe limpiar después en la próxima etapa.
    '''
    data={}
    
    #Indicamos la url
    
   # data['url_prop']= s_prop_arriendo
    
    #Extramos el título
    data['titulo'] = driver.find_element_by_xpath('//h1[@class="ui-pdp-title"]').text
    
    #Extraemos tipo de propiedad
    if 'Departamento' in driver.find_element_by_xpath('//span[@class="ui-pdp-subtitle"]').text:
        data['tipo_propiedad'] = 'Departamento'
    elif 'Casa' in driver.find_element_by_xpath('//span[@class="ui-pdp-subtitle"]').text: 
        data['tipo_propiedad'] = 'Casa'
    else:
        data['tipo_propiedad'] = ''
    
    #Extraemos tiempo de publicación
    data['dias_publicacion'] = driver.find_element_by_xpath('//p[@class="ui-pdp-color--GRAY ui-pdp-size--XSMALL ui-pdp-family--REGULAR ui-pdp-header__bottom-subtitle"]').text 
    
    #Indicamos fecha de hoy
    data['fecha_extracción']=datetime.now().strftime('%Y-%m-%d')
    
    #Extraemos ubicación
    data['ubicacion']= driver.find_element_by_xpath('//*[@id="root-app"]/div/div[3]/div/div[2]/div[2]/div[1]/div/div[1]/div/p').text
    
    #Extraemos valor A
    valor_a = driver.find_element_by_xpath('div', attrs={'class':'ui-pdp-price__second-line'}).find(attrs={'class':'price-tag-fraction'}).text
    data['valor_A']=valor_a
    
    #Extraemos valor B
    valor_b = s_prop_arriendo.find('div', attrs={'class':'ui-pdp-price__subtitles'}).find(attrs={'class':'price-tag-fraction'}).text
    data['valor_B']=valor_b
    
    #Extraemos valor m2 en propiedad
    data['valor_m2_prop']=s_prop_arriendo.find_all('p', attrs={'class':'ui-pdp-color--BLACK ui-pdp-size--XSMALL ui-pdp-family--REGULAR ui-pdp-price-comparison__extra-info-element-value'})[0].text
    
    #Extraemos valor m2 en comuna
    data['valor_m2_comuna']=s_prop_arriendo.find_all('p', attrs={'class':'ui-pdp-color--BLACK ui-pdp-size--XSMALL ui-pdp-family--REGULAR ui-pdp-price-comparison__extra-info-element-value'})[1].text

    #datos de tabla
    datos_tabla = s_prop_arriendo.find('table', attrs={'class':'andes-table'}).find_all('th')
    dato = datos_tabla[0]
    datos = [dato.text for dato in datos_tabla]
    
    #valores de tabla
    valores_tabla = s_prop_arriendo.find('table', attrs={'class':'andes-table'}).find_all('td')
    valor = valores_tabla[0]
    valores = [valor.text for valor in valores_tabla]
    
    tabla = dict(zip(datos, valores))
    
    #Unimos todo
    data.update(tabla)
    
    return data



In [ ]:
def obtener_info_selenium(lista_url_prop):
    hora_start = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
    start_time = time() #Comienza conteo de tiempo
    print(f'El scraping se comienza a ejecutar a las {hora_start}')
    prop = 1 #Nº de propiedad con que comienza la descarga (sólo referencial, no condiciona la programación)
    info = []
    for url in lista_url_prop:
        driver.get(url)
        print(f'Se encontraron {len(lista_url_prop)} propiedades en la lista')#
        print(f'Inicializando scraping de la propiedad nº{prop}...')   
        detalle_arriendo = obtener_info_arriendo(driver)
        print(f'Se termina scraping a propiedad...')
        
    else:
        print(f'No fue posible scrapear la propiedad nº{prop}')
        pass
        prop +=1
        info.append(detalle_arriendo)
        
    print(f'Se escrapearon {prop-1} propiedades')
    hora_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
    print(f'El scraping se comienzó a ejecutar a las {hora_start}')
    print(f'El scraping se termina de ejecutar a las {hora_end}')
    elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
    print("Tiempo de ejecución: %.3f seconds." % elapsed_time)  
    
    return info

In [194]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="ea18bf66dd9a4fcb1f072bf1bff67db6")>>